In [16]:
import json
from copy import deepcopy
with open('dag_stats', "r") as f1:
    last_line = f1.readlines()[-1]
    d = json.loads(last_line)

In [15]:
print('sol: ',d['sol'])
print('bw: ',d['bw'])
print('px: ',d['px'])
print('g: ', d['graph'])

sol:  {'R': [31], 'S': [31], 'K': [0], 'M': [31]}
bw:  {'31': {'31': 0.0, '96': 7.58, '0': 11.15}, '96': {'96': 0.0, '0': 23.61}, '0': {'0': 0.0}}
px:  {'31': 0.016814632983291354, '96': 0.018133857608359397, '0': 0.012490545076556062}
g:  {'K_0': {'children': [], 'parents': ['R_0'], 'node_w': 0, 'edge_w': {}}, 'M_0': {'children': ['R_0'], 'parents': 'S_0', 'node_w': 0.6049999999999667, 'edge_w': {'0': 100}}, 'R_0': {'children': ['K_0'], 'parents': ['M_0'], 'node_w': 0.003999999999892978, 'edge_w': {'0': 102}}, 'S_0': {'children': ['M_0'], 'node_w': 85.97258948605398, 'edge_w': {'0': 1381}}}


In [68]:
#want the final structure to look like
#[('31',{'name':'S_0','node_w':0.5,'edge_w':{'31':3.2}})
# ,('0',_)]
def reformat_edge(level, edgedict, solution):  
        def get_target_node(solution,level,index):
            nextlevel = {'S':'M','M':'R','R':'K'}
            target_node_level = nextlevel.get(level)
            target_node_idx = int(index)
            return str(solution[target_node_level][target_node_idx])
        return {get_target_node(solution, level,k):v 
                for k,v in edgedict.items()}
def reformat_entry(graph_key,graph_value, solution):
    """graph ley value  is like:'K_0', {'children': [], 'parents': ['R_0'], 'node_w': 0, 'edge_w': {'0':1381}}
    solution is like {'R': [31,21], 'S': [31,21], 'K': [0], 'M': [31]}
    result we want it like '31':{'name':'S_0','node_w':0.5,'edge_w':{'31':3.2}}"""
    level, index = graph_key.split('_')
    node = str(solution[level][int(index)])
    new_graph = {'name':graph_key}
    new_graph['node_w'] = graph_value.get('node_w')
    new_graph['edge_w'] = reformat_edge(level, graph_value.get('edge_w'), solution)
    return node, new_graph
def reformat(solution, graph):
    return [reformat_entry(k,v,solution) for k,v in graph.items()]
def nodew_translator(k, entry, px):
    speed = px.get(k)
    new_entry = deepcopy(entry)
    new_entry['node_w'] = new_entry['node_w']/speed
    return new_entry
def translate_nodeweights(px,graph_list):
    """graph_list is like [('0', {'edge_w': {}, 'name': 'K_0', 'node_w': 0}),
     ('31', {'edge_w': {'31': 100}, 'name': 'M_0', 'node_w': 0.6049999999999667})]
     """
    return [(k,nodew_translator(k,v,px)) for k,v in graph_list]
def edgew_translator(source_node, entry, bw):
    edgeweight_dict = entry['edge_w']
    new_edgeweights = {target_node:volume*bw[source_node][target_node] 
                       for target_node,volume
                       in edgeweight_dict.items()}
    new_entry = deepcopy(entry)
    new_entry['edge_w'] = new_edgeweights
    return new_entry
def translate_edgeweights(bw, graph_list):
    return [(k, edgew_translator(k, v, bw)) for k,v in graph_list]
parsed = translate_edgeweights(d['bw'],translate_nodeweights(d['px'],reformat(d['sol'],d['graph'])))
for k,v in parsed:
    print(k,v)

0 {'edge_w': {}, 'node_w': 0.0, 'name': 'K_0'}
31 {'edge_w': {'31': 0.0}, 'node_w': 35.980565296973964, 'name': 'M_0'}
31 {'edge_w': {'0': 1137.3}, 'node_w': 0.23788803501496375, 'name': 'R_0'}
31 {'edge_w': {'31': 0.0}, 'node_w': 5112.962594633178, 'name': 'S_0'}
